# Table of contents:

* [Homomorphisms](#homomorphisms)
    * [Group homomorphism](#homogroups)
* [Homomorphism on the RSA cryptosystem](#RSA)
* [Homomorphism on Paillier cryptosystem](#paillier)
* [Homomorphism on Elgamal cryptosystem](#elgamal)
* [Fully Homomorphic Encryption (FHE)](#fhe)
    
Author: [Sebastià Agramunt Puig](https://github.com/sebastiaagramunt) for [OpenMined](https://www.openmined.org/) Privacy ML Series course.


# Homomorphisms <a class="anchor" id="homomorphisms"></a>

This section is devoted to understand what are homomorphisms and how can we compute on the ciphertext domain sums and multiplications (in some cases!).

## Group homomorphism <a class="anchor" id="homogroups"></a>

Let's put some formalism first. Let $(G, \bigstar)$ and $(H, \square)$ be algebraic groups. A group homomorphism from $G$ to $H$ is a function

$$f: G \rightarrow H$$

such that

$$f(x \bigstar y) = f(x)\square f(y)$$
$$\forall x, y \in G$$

As a simple example, take the two groups $(\mathbb{R}, +)$ and $(\mathbb{R}^+, \cdot)$, i.e. the real numbers with addition group and real positive numbers with multiplication group. Take the function $f(x)=exp(x)$, then $f$ defines an homomorphism with the two groups since

$$f(x+y)=e^{x+y}=e^x\cdot e^y=f(x)\cdot f(y)$$


# Homomorphism on the RSA <a class="anchor" id="RSA"></a>

RSA cryptosystem defines a homomorphism on $(\mathbb{Z}_p, \cdot)$ 

$$Enc: (\mathbb{Z}_N, \cdot) \rightarrow (\mathbb{Z}_N, \cdot)$$


Again, this cryptosystem has been explained in another notebook (I remind you though that here $N$ is a public parameter product of two randomly chosen primes $p$ and $q$).

We will test that:

$$Dec(Enc(m_x)\cdot Enc(m_y)) = m_x\cdot m_y$$


In [1]:
from random import randrange, seed

bits = 64
seed(3)

In [2]:
from crypto import RSAKeyGenerator, RSAEncrypt, RSADecrypt

PublicKeyRSA, PrivateKeyRSA = RSAKeyGenerator(bits)

N = PublicKeyRSA[0]
e = PublicKeyRSA[1]
d = PrivateKeyRSA[1]

mx, my = randrange(N), randrange(N)

print(f"RSA parameters:\n-Public:\n\tN={N}\n\te={e}")
print(f"\n-Private:\n\td={d}")
print(f"Plaintext messages:\nmx={mx}\nmy={my}\n")

RSA parameters:
-Public:
	N=115110739666747791921112868975170199429
	e=105162410562790530939665171758775420585

-Private:
	d=69909657913518582007458109939751496953
Plaintext messages:
mx=8672582414284755974426448173802908311
my=8478698242011685851719386265609182430



In [3]:
mult = mx*my%N
mult2 = RSADecrypt(RSAEncrypt(mx, PublicKeyRSA)*RSAEncrypt(my, PublicKeyRSA)%N, PrivateKeyRSA)


print(f"plaintext multiplication: {mult}")
print(f"ciphertext multiplication and decryption: {mult2}")

plaintext multiplication: 64703202768683605447684872529566588771
ciphertext multiplication and decryption: 64703202768683605447684872529566588771


# Homomorphism on Paillier <a class="anchor" id="paillier"></a>

Paillier cryptosystem defines an homomorphism on the group $(\mathbb{Z}_{N^2}, +)$ where $f$ is the encryption function, this is:

$$Enc: (\mathbb{Z}_{N}, +) \rightarrow (\mathbb{Z}_{N^2}, \cdot)$$

where the function $Enc(x)$ is

$$Enc(x)=g^x \cdot r^N \pmod{N^2}$$

and $N$ is the product of two prime numbers $p$ and $q$ such that the maximum common multiple of $N$ and $(p-1)(q-1)$ is 1. At the same time, $g$ is a random element of $\mathbb{Z}_{N^2}$ and $r$ is a random number the range $0<r<N$ and part of the group ($\mathbb{Z}_N, \cdot$) i.e. $gcm(r, N)=1$.

There is another calculated parameter kept private 

$$\mu=(L(g^{\lambda}\pmod{N^2}))^{-1}\pmod{N}$$

where the function $L$ is defined as

$$L(x)=\frac{x-1}{n}$$

taking the integer part of the divison and $\lambda$ is the least common multiple of the order of the primes i.e. of $p-1$ and $q-1$. 

One can decrypt the ciphertext $c$ and recover the original message $x$ as

$$x=L(c^{\lambda} \pmod{N^2})\cdot \mu \pmod{N}$$

$$Enc(m_x+m_y)=Enc(m_x)\cdot Enc(m_y)$$

Thus the encryption of two plaintexts in $N^2$ is the multipication of their ciphertexts. Let's try to show this in some examples, the proof of the homomorphic encryption can be found [elsewhere](https://doi.org/10.1016/B978-0-12-801595-7.00005-7).

In [4]:
from crypto import PaillierKeyGenerator, PaillierEncrypt, PaillierDecrypt

PublicKeyPaillier, PrivateKeyPaillier = PaillierKeyGenerator(bits)
N = PublicKeyPaillier[0]
g = PublicKeyPaillier[1]

l = PrivateKeyPaillier[1]
mu = PrivateKeyPaillier[2]

mx, my = randrange(N), randrange(N)
print(f"Paillier parameters:\n-Public:\n\tN={N}\n\tg={g}")
print(f"\n-Private:\n\tN={N}\n\tlambda={l}\n\tmu={mu}")
print(f"Plaintext messages:\nm1={mx}\nm2={my}\n")

Paillier parameters:
-Public:
	N=317228669253423891268456764728213811059
	g=64973536771563122426068530836460408091719290602995400805192169229975365662165

-Private:
	N=317228669253423891268456764728213811059
	lambda=158614334626711945616416859266658605752
	mu=49375156409393697151841174487733979265
Plaintext messages:
m1=114661641808165475665698628090987308441
m2=107884585413955112054806888761877394689



We will test that 

$$m_1+m_2 \pmod{N} = Dec(Enc(m_1)\cdot Enc(m_2)\pmod{N^2})$$

on plaintext because the encryption on Paillier is not bijective, i.e. applying the same algorithm on the same input we won't get the same result, that is because it depends on a randomly sampled $r$ (look at the source code of encryption). 

In [5]:
print(f"Sum modulo N of plaintext: {(mx+my)%N}")
prod_ciphertext = PaillierEncrypt(mx, PublicKeyPaillier)*PaillierEncrypt(my, PublicKeyPaillier)%(N**2)
decrypted_prod = PaillierDecrypt(prod_ciphertext, PrivateKeyPaillier)
print(f"Decryption of the product encoded: {decrypted_prod}")

assert decrypted_prod==(mx+my)%N, "something went wrong"

Sum modulo N of plaintext: 222546227222120587720505516852864703130
Decryption of the product encoded: 222546227222120587720505516852864703130


It can also be shown that

$$Dec(Enc(m1)g^{m_2}\pmod{N^2})=m_1+m_2 \pmod{N}$$

In [6]:
decrypted_prod = PaillierDecrypt(PaillierEncrypt(mx, PublicKeyPaillier)*pow(g, my, N**2), PrivateKeyPaillier)

assert (mx+my)%N==decrypted_prod, "something went wrong"
print(f"Sum modulo N of plaintexts: {(mx+my)%N}")
print(f"Decrypted product: {decrypted_prod}")

Sum modulo N of plaintexts: 222546227222120587720505516852864703130
Decrypted product: 222546227222120587720505516852864703130


but this does not define any homomorphism since we are multiplying an exponenciated plaintext (from the first group) to a ciphertext (from the second group). 

# Homomorphism on ElGamal <a class="anchor" id="elgamal"></a>

El Gamal cryptosystem defines a homomorphism on $(\mathbb{Z}_p, \cdot)$ 

$$Enc: (\mathbb{Z}_p, \cdot) \rightarrow (\mathbb{Z}_p, \cdot) \times (\mathbb{Z}_p, \cdot)$$

s. t 

$$Enc(m) = c = (c_1, c_2)$$

This cryptosystem has been explained in another notebook so I refer the reader there to understand it better. The muliplication in the plaintext is defined the usual way but on the ciphertext we multiply pointwise (Shorr):

$$Enc(m_x) = (c_{x1}, c_{x2})$$
$$Enc(m_y) = (c_{y1}, c_{y2})$$
$$Enc(m_x)\cdot Enc(m_y) = (c_{x1}\cdot c_{y1}, c_{x2}\cdot c_{y2})$$

We have to test the homomorphism:

$$Enc(m_x\cdot m_y)=Enc(m_x)\cdot Enc(m_y) = (c_{x1}\cdot c_{y1}, c_{x2} \cdot c_{y2})$$

or equivalently 

$$Dec(Enc(m_x)\cdot Enc(m_y))=m_x\cdot m_y$$

In [7]:
from crypto import ElGamalKeyGenerator, ElGamalEncrypt, ElGamalDecrypt

PublicKeyElGamal, PrivateKeyElGamal = ElGamalKeyGenerator(bits)
A = PublicKeyElGamal[0]
g = PublicKeyElGamal[1]
p = PublicKeyElGamal[2]

sk = PrivateKeyElGamal[0]
mx, my = randrange(p), randrange(p)

print(f"ElGamal parameters:\n-Public:\n\tA={A}\n\tg={g}\n\tp={p}")
print(f"\n-Private:\n\tsk={sk}\n\n")
print(f"Plaintext messages:\nmx={mx}\nmy={my}\n")

ElGamal parameters:
-Public:
	A=6449297829066993208
	g=1588082161193385932
	p=9628098341074011907

-Private:
	sk=4567125481059076332


Plaintext messages:
mx=8024907043293506373
my=9224214575254058992



In [8]:
c1 = ElGamalEncrypt(mx, PublicKeyElGamal)
c2  = ElGamalEncrypt(my, PublicKeyElGamal)

# plaintext multiplicatoin
mxmy = mx*my%p

# cipher multiplication
cipher_mult = (c1[0]*c2[0]%p, c1[1]*c2[1]%p)
dec_cipher_mult = ElGamalDecrypt(cipher_mult, PrivateKeyElGamal)

assert mxmy==dec_cipher_mult, "something went wrong"

# print stuff
print(f"ciphertext 1 = {c1}\nciphertext 2 = {c2}\n")
print(f"Product of m1 and m2 in ciphertext = {cipher_mult}")
print(f"Decrypting the product of ciphertext {dec_cipher_mult}")
print(f"Product of the plaintexts {mxmy}")

ciphertext 1 = (5565390317215748065, 6867623251151918661)
ciphertext 2 = (6806882144077333449, 2465824210384759077)

Product of m1 and m2 in ciphertext = (5849767633442321908, 7920321264608353552)
Decrypting the product of ciphertext 7085892333900747554
Product of the plaintexts 7085892333900747554


Up until now we have seen that RSA, ElGamal and Paillier encryption schemes are homomorphic with a certain operation. Furthermore this operation can be applied to all elements of the group (i.e. no matter what are the two oringinal messages as long as they belong to the initial group set). This properties defines what we know as **partial homomorphic encryption**.

## Fully homomorphic encryption <a class="anchor" id="fhe"></a>

Ideally we would like to define encryption functions such that they do not only work in one operation (say sum or product) but with the two at the same time. This is what the community coins as fully **homomorphic encryption**. During many years there was a quest to find a fully homomorphic encryption scheme and finally [Craig Gentry](https://en.wikipedia.org/wiki/Craig_Gentry_(computer_scientist)) published his PhD thesis in 2009 with the first fully homomorphic encryption scheme based on [algebraic lattices](https://en.wikipedia.org/wiki/Lattice-based_cryptography).